In [1]:
import requests
import bs4
import pandas as pd
from urllib.parse import urlparse, urljoin
import ngram_value
import size

In [2]:
url = input('Enter the url :')

Enter the url :www.exeterpremedia.com


In [3]:
# adds (http://)  if not included
if 'http://' in url or 'https://' in url:
    pass
else:
    url = 'http://' + url    

In [4]:
domain_name = urlparse(url).netloc   # returns the domain name of the website

In [5]:
#checks whether link is valid or not
def check_url(url):
    try:
        response = requests.get(url)
        #print("URL is valid and exists on the Internet")
        return True
    except:
        #print("URL does not exist on the Internet")
        return False

In [6]:
check_url(url)

True

In [7]:
# returns the homepage of the web_link
def home_page(url):
    count = 0
    url = url + '/'
    url_len = len(url)
    for i in range(url_len):
        if '/' == url[i]:
            count +=1
            if count == 3:
                break
    return i    

In [8]:
val = home_page(url)
url = url[0:val]

In [9]:
#finding all the links 

valid_link = set()  # set variable to remove duplicate values

def link_gen(url):
    
    res = requests.get(url)
    soup = bs4.BeautifulSoup(res.text,'lxml')
    
    for link in soup.find_all('a',href=True):                            # loop through all links
        
        first_char = link['href'][0]
        
        if (check_url(link['href']) and  '#' not in link['href'][0]):    # excluding garbage link
            if first_char != '/':
                valid_link.add(link['href'])
                
        
        if first_char == '/':                                            # adding url to few internal link                                      
            final_link = url + link['href']
            valid_link.add(final_link)
            
                

In [10]:
link_gen(url)

In [11]:
valid_num = len(valid_link)  # no of valid links

In [12]:
# necessary list variables to store data in csv

internal_links = []
external_links = []

title_tag = []
meta_value = []
internal_value = []
external_value = []
uni_value = []
bi_value = []
size_value = []




def get_title(valid_link):
    
    for link in valid_link:                                 # looping through all valid link
        
        try:
            res = requests.get(link)                        # try except block if it throws error
            soup = bs4.BeautifulSoup(res.text,'lxml')
        except:
            continue
        
        
        if len(soup.select('title')) != 0:                 # selecting title of the webpage and storing it in list
            title_tag.append(soup.select('title')[0].text)
        else:
            title_tag.append('N/A')
        
        
        description = soup.find('meta', attrs={'name':'og:description'}) or soup.find('meta', attrs={'property':'description'}) or soup.find('meta', attrs={'name':'description'})
        if description:                                   # adding desc from meta tag to list
           description_text = description.get('content')
        else:
            description_text = 'N/A'
            
        meta_value.append(description_text)
        
        
        
        if domain_name in link:                           # classfying the internal and external links                      
            internal_links.append(link)
            internal_value.append('yes')
            external_value.append('no')
        else:
            external_links.append(link)
            internal_value.append('no')
            external_value.append('yes')
        
        a,b = ngram_value.uni_bi_values(link)            # appending unigrams,bigrams
        uni_value.append(a)
        bi_value.append(b)
        
        
        temp = size.total_size(link)                    # adding size of the web page to list
        size_value.append(temp)
                  
    

In [13]:
get_title(valid_link)

In [14]:
new_valid_link = list(valid_link)   # set to list

In [15]:
final_result = {}

for i in range(valid_num):         # looping through each valid link and storing coresponding values
    
    final_result[i] = [new_valid_link[i],title_tag[i],meta_value[i],internal_value[i],external_value[i],uni_value[i],bi_value[i],size_value[i]] 

In [16]:
final_result_df = pd.DataFrame.from_dict(final_result,orient='index', columns=['Links','Title','Meta-data(DESC)','Internal','External','unigram','bi-gram','web-page size(KB)'])

In [21]:
final_result_df.tail(5)    #printing Df

,Links,Title,Meta-data(DESC),Internal,External,unigram,bi-gram,web-page size(KB)
80,https://www.linkedin.com/feed/hashtag/?keyword...,Sign Up | LinkedIn,500 million+ members | Manage your professiona...,no,yes,"[Services, We, services, US, us, publishers, E...","[(Personal, Information), (Exeter, Premedia), ...",24.3701
81,https://www.exeterpremedia.com/contact-us/,Exeter - Contact Us,Reach us at ravi@exeterpremedia.com and +91-44...,yes,no,"[Services, services, We, US, us, publishers, E...","[(Personal, Information), (Exeter, Premedia), ...",104.938
82,https://www.exeterpremedia.com/wp-content/uplo...,N/A,N/A,yes,no,"[Services, services, We, US, us, publishers, E...","[(Personal, Information), (Exeter, Premedia), ...",4.67578
83,http://www.exeterpremedia.com/services/#Editor...,Exeter - Services,Exeter provides editorial services such copyed...,yes,no,"[Services, services, We, US, us, publishers, E...","[(Personal, Information), (Exeter, Premedia), ...",126.92
84,https://www.exeterpremedia.com/wp-content/uplo...,N/A,N/A,yes,no,"[Services, services, We, US, us, publishers, E...","[(Personal, Information), (Exeter, Premedia), ...",8.20508


In [ ]:
final_result_df.to_csv('exeter.csv')    #downloading csv file

In [22]:
print('No of Top level pages: ' + str(valid_num))  # total top level pages

No of Top level pages: 85


In [23]:
print('No of Internal links: ' + str(len(internal_links)))

No of Internal links: 51


In [24]:
print('No of Internal links: ' + str(len(external_links)))

No of Internal links: 34
